In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
import json 
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy
csvfile = "Buoy Stations - Sheet1.csv" 

In [2]:
buoy_df_raw = pd.read_csv(csvfile)
buoy_df = buoy_df_raw.dropna()
buoy_df.head()

,Buoy,Lat,Long
0,42002,26.055,-93.646
1,42003,25.925,-85.615
2,khhv,26.939,-94.689
3,kemk,27.819,-94.323
4,42019,27.910,-95.345


In [3]:
lat_val = []
lng_val = []
for index,row in buoy_df.iterrows():
    lat_val.append(row["Lat"])
    lng_val.append(row["Long"])

[26.055, 25.925, 26.939, 27.819, 27.91, 27.354, 27.625, 26.968, 25.9, 29.232, 28.982, 28.351, 28.842, 29.635, 29.087, 27.774, 28.314, 22.124, 19.82, 27.463, 23.822, 23.37, 30.06, 28.787, 28.501, 27.505, 27.345, 28.501, 27.59, 25.701, 25.171, 14.394, 26.01, 27.173, 15.287, 16.433, 17.591, 17.695, 18.094, 17.869, 21.03]
[-93.646, -85.615, -94.689, -94.323, -95.345, -94.625, -90.441, -96.693, -89.287, -94.413, -94.899, -96.006, -94.242, -93.642, -92.506, -96.972, -95.62, -93.941, -84.945, -71.466, -68.384, -67.517, -87.548, -86.007, -84.508, -83.741, -84.275, -84.508, -82.931, -83.65, -83.475, -74.816, -83.086, -82.924, -67.477, -63.331, -61.821, -64.754, -65.471, -66.532, -64.79]


In [9]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

[1630818000, 1630731600, 1630645200, 1630558800, 1630472400]

In [54]:
#lists for buoy data dataframe
hour_temp =[]
hour_pressure =[]
hour_wind_speed =[]
hour_wind_deg =[]
hour_time=[]
hour_lat=[]
hour_lng = []

In [56]:
for lat in range(len(lat_val)):
    for x in unix_time():
            historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat_val[lat]}&lon={lng_val[lat]}&dt={x}&appid={weather_api_key}&units=imperial"
            hist_json = requests.get(historical_url).json()
            for i in range((len(hist_json["hourly"]))):
                hour_temp.append(hist_json["hourly"][i]["temp"])
                hour_pressure.append(hist_json["hourly"][i]["pressure"])
                hour_wind_speed.append(hist_json["hourly"][i]["wind_speed"])
                hour_wind_deg.append(hist_json["hourly"][i]["wind_deg"])
                hour_time.append(hist_json["hourly"][i]["dt"])
                hour_lat.append(hist_json["lat"])
                hour_lng.append(hist_json["lon"])
      
        

In [62]:
buoy_dict = {"Lat":hour_lat,
            "Long":hour_lng,
            "Temp":hour_temp,
            "Pressure":hour_pressure,
            "Wind_Speed":hour_wind_speed,
            "Wind_Degrees":hour_wind_deg,
            "Unix Time":hour_time}
buoy_output_data = pd.DataFrame(buoy_dict)
buoy_output_data

,Lat,Long,Temp,Pressure,Wind_Speed,Wind_Degrees,Unix Time
0,26.055,-93.646,84.87,1013,6.11,90,1630800000
1,26.055,-93.646,84.76,1014,6.60,89,1630803600
2,26.055,-93.646,84.60,1014,6.49,95,1630807200
3,26.055,-93.646,84.60,1014,6.87,104,1630810800
4,26.055,-93.646,84.38,1014,7.07,115,1630814400
...,...,...,...,...,...,...,...
575,27.910,-95.345,86.04,1010,12.35,188,1630522800
576,27.910,-95.345,85.55,1010,12.53,184,1630526400
577,27.910,-95.345,85.89,1010,12.30,182,1630530000
578,27.910,-95.345,85.91,1009,10.18,183,1630533600


In [31]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [32]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [65]:
df_city

,City,lat,lng
0,austin,30.253810,-97.857107
1,natchitoches,31.626140,-93.775218
2,huntsville,31.069324,-95.576204
3,port arthur,29.263067,-93.530487
4,pleasanton,28.936713,-98.573205
...,...,...,...
6495,lufkin,30.874091,-94.583858
6496,huntsville,30.611970,-95.810950
6497,huntsville,30.600513,-95.919274
6498,bryan,30.521264,-96.621677


In [34]:
#check length of cities
# len(list(df_city["City"].unique()))

111

In [64]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

In [44]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    hour_temp_city =[]
    hour_pressure_city =[]
    hour_wind_speed_city =[]
    hour_wind_deg_city=[]
    hour_time_city=[]
    city_name=[]
    country_code=[]
    for city in target_cities:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},Texas,USA&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]

        except:
            print("City not found. Skipping...")
            pass
        if response1["sys"]["country"]=="US":
            for x in unix_time():
                historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
                hist_json = requests.get(historical_url).json()
                for i in range(len(hist_json["hourly"])):
                    hour_temp_city.append(hist_json["hourly"][i]["temp"])
                    hour_pressure_city.append(hist_json["hourly"][i]["pressure"])
                    hour_wind_speed_city.append(hist_json["hourly"][i]["wind_speed"])
                    hour_wind_deg_city.append(hist_json["hourly"][i]["wind_deg"])
                    hour_time_city.append(hist_json["hourly"][i]["dt"]
                    city_name.append(response1['name'])
                    country_code.append(response1["sys"]["country"])
                
    d=dict()
    d["City Name"] = city_name
    d["Country"] = country_code
    d["Temperature"] = hour_temp_city
    d["Pressure"] = hour_pressure_city
    d["Wind Degrees"]=hour_wind_deg_city
    d["Wind_Speed"] = hour_wind_speed_city
    return(d)

In [ ]:
df2 =pd.DataFrame(find_location(target_cities))

In [63]:
df2

,City Name,Country,Temperature,Pressure,Wind Degrees,Wind_Speed
0,Austin,US,93.67,1009,90,1.99
1,Austin,US,90.68,1009,90,1.01
2,Austin,US,88.45,1010,135,1.99
3,Austin,US,86.90,1011,135,1.99
4,Austin,US,85.17,1013,135,3.00
...,...,...,...,...,...,...
575,Pleasanton,US,71.87,1009,190,1.99
576,Pleasanton,US,74.25,1009,126,1.01
577,Pleasanton,US,74.12,1009,78,1.01
578,Pleasanton,US,75.76,1009,258,1.99
